# RAG Workbench - LlamaIndex Function Calls

In this notebook, we showcase how to use the LastMile Tracing SDK and [RAG Workbench](https://rag.lastmileai.dev/) to manage, evaluate, and debug function calls with LlamaIndex.

## Notebook Outline
* [Step 1: Install and Setup](#install)
* [Step 2: Configure LastMile Instrumnetor](#trace)
* [Step 3: Write Function Calls](#function)
* [Step 4: Ask a Question](#question)
* [Step 5: View Results in RAG Workbench UI](#ui)

<a name="install"></a>
## Step 1: Install and Setup

First install the required packages.

In [ ]:
%pip install llama-index-agent-openai
%pip install llama-index-llms-openai
!pip install "tracing-auto-instrumentation[llama-index]" --upgrade

Import the modules used in this tutorial.

In [2]:
import llama_index.core
from tracing_auto_instrumentation.llama_index import LlamaIndexCallbackHandler
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool
from getpass import getpass

We also need the following API tokens/keys:

* **LastMile AI API Token:** Go to the [LastMile Settings page](https://lastmileai.dev/settings?page=tokens). You will need to first create a LastMile AI account.
* **OpenAI API Key:** Go to [OpenAI API Keys page](https://platform.openai.com/account/api-keys) to create and access your OpenAI API Key.

Run the code cell below after setting the keys either in **Google Colab Secrets** or in `.env` in your directory. Avoid inputting keys directly in the notebook.

In [3]:
import os

try:
    # If running on Google Colab, use userdata to securely input keys
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    LASTMILE_API_TOKEN = userdata.get('LASTMILE_API_TOKEN')
except ModuleNotFoundError:
    # If running locally, load keys from .env file
    from dotenv import load_dotenv
    load_dotenv()
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    LASTMILE_API_TOKEN = os.getenv('LASTMILE_API_TOKEN')

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['LASTMILE_API_TOKEN'] = LASTMILE_API_TOKEN

<a name="trace"></a>

## Step 2: Configure LastMile Instrumentor
First, we configure the LastMile Instrumentor which will auto-instrument tracing for your LlamaIndex application. This allows us to easily see the various steps in our RAG system in the RAG Workbench UI. 

In [4]:
llama_index.core.global_handler = LlamaIndexCallbackHandler(
    project_name="LlamaIndex-Function-Call",
)

<a name="function"></a>

## Step 3: Write Function Calls
For this example, we will use 2 simple function calls (multiple and add).

In [5]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)

llm = OpenAI(model="gpt-3.5-turbo-1106")
agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool], llm=llm, verbose=True
)

<a name="question"></a>

## Step 4: Ask a Question
Let's now ask a question and see the function calls execute appropriately.

In [6]:
response = agent.chat("What is (121 * 3) + 42?")
print(str(response))

Added user message to memory: What is (121 * 3) + 42?
=== Calling Function ===
Calling function: multiply with args: {"a": 121, "b": 3}
Got output: 363

=== Calling Function ===
Calling function: add with args: {"a": 363, "b": 42}
Got output: 405

The result of (121 * 3) is 363, and when you add 42 to that, you get 405.


<a name="ui"></a>

## Step 5: View Results in RAG Workbench UI

We can view the results in the RAG Workbench UI.

From your terminal, export your LASTMILE_API_TOKEN

```bash
export LASTMILE_API_TOKEN="<your-api-token>"
```

Next, run the following command in your terminal to launch the UI:

```bash
rag-debug launch
```

Navigate to the url provided by the RAG Workbench (opens up your web browser). This will look like http://localhost:8080/

1. Click the **Traces Tab**.
2. Select Project 'LlamaIndex-Function-Call'. You can see the Trace logged in the UI.

<img width="700" alt="traces_llfc" src="https://github.com/lastmile-ai/aiconfig/assets/81494782/d79fe9eb-2c04-4b11-83d2-0447727c99f4">




3. Click on a Trace to see the function calls executed and other metadata.

<img width="700" alt="trace_overview_llfc" src="https://github.com/lastmile-ai/aiconfig/assets/81494782/b4d20aa3-1e5a-46a5-b99a-31172bb46a24">



